# Compute Isothermal Spectra

We need to compute synthetic spectra and continuum for our full bandpass as a function of temperature 

In [5]:
import sys

import hissw

sys.path.append('../../scripts/')
from response import Channel, SpectrogramChannel

In [6]:
chan = SpectrogramChannel(0, '../data/MOXSI_effarea.genx')

In [10]:
chan.wavelength

<Quantity [ 1.   ,  1.055,  1.11 , ..., 59.85 , 59.905, 59.96 ] Angstrom>

In [9]:
ssw = hissw.Environment(ssw_packages=['chianti'])

In [23]:
script = '''
ioneq_name='/home/jovyan/ssw/packages/chianti/dbase/ioneq/chianti.ioneq'
wmin=1
wmax=2
logt=[5.0, 6.0]
pressure=1e+15
density=pressure/logt

;generate transition structure for selected wavelength and temperature range
ch_synthetic, wmin, wmax, ioneq_name=ioneq_name, output=transitions,$
              logt_isothermal=logt,$
              density=density,$
              /no_sum_int
              
;compute the spectra as a function of lambda and T
make_chianti_spec, transitions, 
'''

In [24]:
output = ssw.run(script)

IDL 8.8.1 (linux x86_64 m64).
(c) 2021, Harris Geospatial Solutions, Inc.
% Error initializing graphics device GL.

Licensed for use by: NASA - Goddard Space Flight Center - MAIN
License: MNT-5527480
A new version is available: IDL 8.8.2
https://harrisgeospatial.flexnetoperations.com
 
% Compiled module: USE_CHIANTI.
% Compiled module: DIR_EXIST.
% DEVICELIB: Added system variable !BCOLOR
% DEVICELIB: Added system variable !ASPECT
% Compiled module: HISSW_PROCEDURE.
% Compiled module: CH_SYNTHETIC.
% Compiled module: UNIQ.
% Compiled module: DEFAULT.
% Compiled module: DATATYPE.
% Compiled module: READ_IONEQ.
% Compiled module: READ_MASTERLIST.
% Compiled module: CONVERTNAME.
% Compiled module: STR_SEP.
% Compiled module: ION2SPECTROSCOPIC.
% Compiled module: ZION2FILENAME.
% Compiled module: SPLINE.
% Compiled module: CH_SETUP_ION.
% Compiled module: ION2FILENAME.
% Compiled module: READ_WGFA_STR.
% Compiled module: READ_ELVLC.
% Compiled module: ZION2SPECTROSCOPIC.
% Compiled module:

In [40]:
foo = output['transitions'][0]['lines'][0]

In [44]:
output['transitions'][0].dtype

dtype((numpy.record, [(('lines', 'LINES'), 'O'), (('ioneq_logt', 'IONEQ_LOGT'), 'O'), (('ioneq_name', 'IONEQ_NAME'), 'O'), (('ioneq_ref', 'IONEQ_REF'), 'O'), (('wvl_limits', 'WVL_LIMITS'), 'O'), (('model_file', 'MODEL_FILE'), 'O'), (('model_name', 'MODEL_NAME'), 'O'), (('model_ne', 'MODEL_NE'), 'O'), (('model_pe', 'MODEL_PE'), 'O'), (('model_te', 'MODEL_TE'), 'O'), (('wvl_units', 'WVL_UNITS'), 'O'), (('int_units', 'INT_UNITS'), 'O'), (('add_protons', 'ADD_PROTONS'), '>i2'), (('date', 'DATE'), 'O'), (('version', 'VERSION'), 'O'), (('lookup', 'LOOKUP'), '>i2'), (('photoexcitation', 'PHOTOEXCITATION'), '>i2'), (('logt_isothermal', 'LOGT_ISOTHERMAL'), 'O'), (('logem_isothermal', 'LOGEM_ISOTHERMAL'), 'O')]))

In [35]:
output['transitions'][0]['logem_isothermal']

array([0., 0.], dtype=float32)

In [33]:
foo

b'erg cm-2 sr-1 s-1'